In [21]:
import pandas as pd
import numpy as np
from scipy import stats as st
from pprint import pprint as print
import snoop
from IPython.core.interactiveshell import InteractiveShell
from scipy.stats import mannwhitneyu
from statsmodels.stats.proportion import proportion_confint

InteractiveShell.ast_node_interactivity = "all"

Во-первых, для меня это возможность углубить свой преподавательский опыт и поделиться уже имеющимся, плюс мне очень интересно ознакомиться с программой Яндекс.Практикума. В дальнейшем хотелось бы ревьюить также работы с потока не только аналитиков, но и дата-сайентистов.

1. Определите пересекающуюся аудиторию в рамках групп treatment и control. Какой размер пересекающейся аудитории?

In [36]:
df1, df2 = pd.read_csv('ab_data.csv'), pd.read_csv('revenue_data.csv')
df1
df2

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


,user_id,revenue
0,851104,10.284548
1,804228,7.649273
2,661590,11.909644
3,853541,23.693355
4,864975,19.996224
...,...,...
290579,751197,17.600421
290580,945152,10.936073
290581,734608,8.701321
290582,697314,19.623677


In [31]:
t, c = set(df1[df1['group'] == 'treatment']['user_id']), set(
    df1[df1['group'] == 'control']['user_id'])
intersection = t & c
len(t & c)


1895

In [37]:
df1.set_index('user_id').drop(intersection, axis=0, inplace=True)
df1[df1['group'] == 'control'].nunique()
tt = df1.reset_index().groupby(['user_id']).count()['timestamp']
# tt
len(tt[tt == 2])

user_id         146195
timestamp       147202
group                1
landing_page         2
converted            2
dtype: int64

3894

In [40]:
df1df2 = df1.merge(df2, how='left')
cnt, trt = df1df2[df1df2['group'] == 'control'][[
    'revenue', 'group'
]], df1df2[df1df2['group'] == 'treatment'][['revenue', 'group']]


In [42]:
df1['group'] = df1['group'].apply(lambda x: 1 if x == 'treatment' else 0)


In [43]:
df1

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,0,old_page,0
1,804228,2017-01-12 08:01:45.159739,0,old_page,0
2,661590,2017-01-11 16:55:06.154213,1,new_page,0
3,853541,2017-01-08 18:28:03.143765,1,new_page,0
4,864975,2017-01-21 01:52:26.210827,0,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,0,old_page,0
294474,945152,2017-01-12 00:51:57.078372,0,old_page,0
294475,734608,2017-01-22 11:45:03.439544,0,old_page,0
294476,697314,2017-01-15 01:20:28.957438,0,old_page,0


In [6]:
mannwhitneyu(cnt, trt, alternative='two-sided')

MannwhitneyuResult(statistic=10868722457.5, pvalue=0.20768055622934778)

In [8]:
t = df1.set_index('user_id').join(df2.set_index('user_id'), on='user_id')
t.reset_index()

,user_id,timestamp,group,landing_page,converted,revenue
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,10.284548
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,7.649273
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,11.909644
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,23.693355
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,19.996224
...,...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0,17.600421
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0,10.936073
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0,8.701321
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0,19.623677


In [15]:
import scipy

scipy.stats.mannwhitneyu(t[t["group"] == "treatment"]["revenue"],
                         t[t["group"] == "control"]["revenue"],
                         alternative='two-sided')


MannwhitneyuResult(statistic=10810599294.5, pvalue=0.20768055622934778)

Во-первых, Series это одномерная структура данных представляющая из себя набор пар "ключ-значение", где ключи могут выступать в роли индексов. Если объединить (поставить стопкой друг на друга) несколько series с одинаковыми ключами (индексами) то получится DataFrame, двумерная структура данных – таблица, в которой бывшие индексы будут названиями столбцов. 
https://dsc.cloud/999174/pb-NNpRObM3R0.png

In [27]:
df1 = df1.set_index('user_id').drop(intersection, axis=0)
# len(df1)
df1 = df1.reset_index()

Очистите данные от сессий пользователей, которые входят в пересекающуюся аудиторию. Все дальнейшие манипуляции производите с очищенными данными

Какое количество уникальных пользователей в группе control?

In [28]:
df1[df1['group'] == 'control'].nunique()


user_id         144300
timestamp       145307
group                1
landing_page         2
converted            2
dtype: int64

Какое количество пользователей совершили более чем 1 сессии за время наблюдений?

In [3]:
g = df1.reset_index()
g = g.groupby(['user_id']).count()['timestamp']
g.value_counts()

1    286690
2      3894
Name: timestamp, dtype: int64

Конверсия рассчитывается как отношение количества сессий к количеству сессий, в которых произошла конверсия (колонка converted). Какая нижняя граница доверительного интревала для конверсии при условии, что alpha = 0.05? (Ответ дайте с точностью до 3 знаков после запятой, используйте математическое округление. Ответ - число в виде десятичной дроби, а не процентного числа)

In [20]:
# mean_conv = len(df1[df1['converted']==1])/len(df1)
len(df1[df1['converted'] == 1]['user_id'].unique()) / len(
    df1['user_id'].unique())


0.12104245244060237

In [14]:
proportion_confint(sum(df1['converted']), len(df1), method='wilson')

(0.11849190251342853, 0.12083640754359773)

In [38]:
def proportions_confint_diff_rel(sample1, sample2, alpha=0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)

    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])

    left_boundary = float(f - g) / n - z * np.sqrt(
        float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n + z * np.sqrt(
        float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [49]:
df1[df1['group'] == 1]['converted']

2         0
3         0
6         1
8         1
9         1
         ..
294462    0
294465    0
294468    0
294472    0
294477    0
Name: converted, Length: 147276, dtype: int64

In [47]:
df1

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,0,old_page,0
1,804228,2017-01-12 08:01:45.159739,0,old_page,0
2,661590,2017-01-11 16:55:06.154213,1,new_page,0
3,853541,2017-01-08 18:28:03.143765,1,new_page,0
4,864975,2017-01-21 01:52:26.210827,0,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,0,old_page,0
294474,945152,2017-01-12 00:51:57.078372,0,old_page,0
294475,734608,2017-01-22 11:45:03.439544,0,old_page,0
294476,697314,2017-01-15 01:20:28.957438,0,old_page,0


In [51]:
proportions_confint_diff_rel(df1[df1['group'] == 0]['converted'],
                             df1[df1['group'] == 1]['converted'])


(-0.0008730438535730722, 0.0038213733599656487)

In [16]:
df3 = pd.read_csv('movie_metadata.csv')

Загрузите датасет movie_metadata.csv, который содержит данные о фильмах из базы IMDb (Internet Movie Database). Колонка duration - данные о длительности фильма.

Сколько пропущенных значений содержится в этой колонке?

In [5]:
len(df3['duration']) - len(df3['duration'].dropna())

15

In [17]:
df3.duration.fillna(value=np.mean(df3.duration), inplace=True)

Создайте колонку movie_duration_category, которая содержала бы 3 категории в зависимости от длительности фильма:

Категория "1. < 90", если фильм длится менее 90 минут;
Категория "2. 90-120", если фильм длится от 90 минут до 2 часов (включительно);
Категория "3. > 120", если фильм длится больше 2 часов.
Постройте сводную таблицу для фильмов с годом выпуска после 2000 (включительно), которая отражала бы количество фильмов:

В строках таблицы - год;
В столбцах таблицы - категория длительности фильма ("< 90", "90-120", "> 120");
Год фильма должен быть представлен в формате YYYY
Сколько фильмов с длительностью от 90 минут до 2 часов было выпущено в 2008 году?

In [18]:
df3['duration'].mean()

107.20107398568018

In [9]:
df3['mov_dur_cat'] = pd.cut(df3.duration, bins=[-np.inf, 90, 120, np.inf])

In [21]:
t = df3.groupby(['title_year', 'mov_dur_cat']).count()
# t[t['title_year']==2008]
t[-30:-20]

color  director_name  num_critic_for_reviews  \
title_year mov_dur_cat                                                   
2007.0     (-inf, 90.0]      37             37                      35   
           (90.0, 120.0]    124            124                     123   
           (120.0, inf]      43             43                      43   
2008.0     (-inf, 90.0]      42             42                      41   
           (90.0, 120.0]    147            147                     147   
           (120.0, inf]      36             36                      36   
2009.0     (-inf, 90.0]      55             55                      53   
           (90.0, 120.0]    164            165                     164   
           (120.0, inf]      40             40                      39   
2010.0     (-inf, 90.0]      34             34                      32   

                          duration  director_facebook_likes  \
title_year mov_dur_cat                                        
2007.0     (-inf, 90.0]         37                       37   
           (90.0, 120.0]       124                      124   
           (120.0, inf]         43                       43   
2008.0     (-inf, 90.0]         42                       42   
           (90.0, 120.0]       147                      147   
           (120.0, inf]         36                       36   
2009.0     (-inf, 90.0]         55                       55   
           (90.0, 120.0]       165                      165   
           (120.0, inf]         40                       40   
2010.0     (-inf, 90.0]         34                       34   

                          actor_3_facebook_likes  actor_2_name  \
title_year mov_dur_cat                                           
2007.0     (-inf, 90.0]                       37            37   
           (90.0, 120.0]                     124           124   
           (120.0, inf]                       43            43   
2008.0     (-inf, 90.0]                       41            42   
           (90.0, 120.0]                     147           147   
           (120.0, inf]                       36            36   
2009.0     (-inf, 90.0]                       55            55   
           (90.0, 120.0]                     165           165   
           (120.0, inf]                       40            40   
2010.0     (-inf, 90.0]                       34            34   

                          actor_1_facebook_likes  gross  genres  ...  \
title_year mov_dur_cat                                           ...   
2007.0     (-inf, 90.0]                       37     25      37  ...   
           (90.0, 120.0]                     124    112     124  ...   
           (120.0, inf]                       43     42      43  ...   
2008.0     (-inf, 90.0]                       42     34      42  ...   
           (90.0, 120.0]                     147    132     147  ...   
           (120.0, inf]                       36     35      36  ...   
2009.0     (-inf, 90.0]                       55     39      55  ...   
           (90.0, 120.0]                     165    139     165  ...   
           (120.0, inf]                       40     37      40  ...   
2010.0     (-inf, 90.0]                       34     21      34  ...   

                          movie_imdb_link  num_user_for_reviews  language  \
title_year mov_dur_cat                                                      
2007.0     (-inf, 90.0]                37                    35        37   
           (90.0, 120.0]              124                   124       123   
           (120.0, inf]                43                    43        43   
2008.0     (-inf, 90.0]                42                    41        42   
           (90.0, 120.0]              147                   147       147   
           (120.0, inf]                36                    36        36   
2009.0     (-inf, 90.0]                55                    55        55   
           (90.0, 120.0]     

In [19]:
df3['budget'] = df3.budget.apply(lambda x: x[:-1])
np.mean(df3['budget'].astype('float'))

35874316.003569305

In [28]:
# df3['plot_keywords'][0].split('|')
# @snoop
def foo(keywords):
    if keywords is not df3.plot_keywords[4]:
        keywords = keywords.split('|')
        if 'love' in keywords and 'death' in keywords: return 'love_and_death'
        elif 'love' in keywords: return 'love'
        elif 'death' in keywords: return 'death'
        else: return 'other'
    else:
        return 'other'


df3['mov_cat'] = df3['plot_keywords'].apply(lambda x: foo(x))

In [29]:
df3.groupby('mov_cat').mean()['imdb_score']

mov_cat
death             6.508130
love              6.533862
love_and_death    6.677778
other             6.436298
Name: imdb_score, dtype: float64

In [5]:
df3[df3['mov_cat'] == 'love']


,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes,mov_cat
9,Color,David Yates,375.0,153.0,282.0,10000.0,Daniel Radcliffe,25000.0,301956980.0,Adventure|Family|Fantasy|Mystery,...,English,UK,PG,250000000.0$,2009.0,11000.0,7.5,2.35,10000,love
26,Color,James Cameron,315.0,194.0,0.0,794.0,Kate Winslet,29000.0,658672302.0,Drama|Romance,...,English,USA,PG-13,200000000.0$,1997.0,14000.0,7.7,2.35,26000,love
104,Color,Jon Turteltaub,235.0,109.0,226.0,370.0,Omar Benson Miller,12000.0,63143812.0,Action|Adventure|Family|Fantasy,...,English,USA,PG,150000000.0$,2010.0,418.0,6.1,2.35,11000,love
233,Color,James L. Brooks,168.0,121.0,274.0,157.0,Domenick Lombardozzi,273.0,30212620.0,Comedy|Drama|Romance,...,English,USA,PG-13,120000000.0$,2010.0,216.0,5.3,1.85,0,love
309,Color,Joel Schumacher,144.0,121.0,541.0,680.0,Rene Auberjonois,920.0,184031112.0,Action|Adventure|Fantasy,...,English,USA,PG-13,100000000.0$,1995.0,710.0,5.4,1.85,0,love
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4882,Color,Dan Curtis,NaN,99.0,45.0,224.0,Campbell Scott,1000.0,NaN,Fantasy|Romance,...,English,USA,Unrated,0.0$,1998.0,393.0,7.4,1.33,515,love
4930,Color,Gary Winick,91.0,78.0,56.0,184.0,Aaron Stanford,376.0,2882062.0,Comedy|Drama|Romance,...,English,USA,PG-13,150000.0$,2000.0,346.0,6.3,1.85,132,love
4942,Color,Paul Schrader,130.0,93.0,261.0,697.0,Ruby Dee,783.0,NaN,Fantasy|Horror|Thriller,...,English,USA,R,18000000.0$,1982.0,782.0,6.1,1.85,0,love
4984,Color,Edward Burns,36.0,98.0,0.0,73.0,Michael McGlone,138.0,10246600.0,Comedy|Drama|Romance,...,English,USA,R,25000.0$,1995.0,111.0,6.6,1.85,265,love


Даны две случайные величины X и Y. Про них известно, что:
E X = 0.5, D X = 2
E Y = 7, D Y = 3.5
cov (X, Y) = -0.8
Найдите дисперсию случайной величины Z = 2 X - 3 Y (верное решение будет между 45 и 50).

$$Var_{X-Y}=Var^2_X+Var^2_Y-2Cov_{XY}$$
$$Var_X=E[X^2]-E[X]^2$$


$$
\begin{aligned}
&\operatorname{Var}(a X+b Y)=a^{2} \operatorname{Var}(X)+b^{2} \operatorname{Var}(Y)+2 a b \operatorname{Cov}(X, Y) \\
&\operatorname{Var}(a X-b Y)=a^{2} \operatorname{Var}(X)+b^{2} \operatorname{Var}(Y)-2 a b \operatorname{Cov}(X, Y)
\end{aligned}
$$

In [39]:
import sympy
from sympy import Symbol as sym

var_x, var_y = sympy.Symbol("var_x"), sympy.Symbol("var_y")
var_z = sympy.Symbol("var_z")
a, b = sympy.Symbol("a"), sympy.Symbol('b')
cov_xy = sympy.Symbol("cov_xy")

In [36]:
E_X2, E2_X = sym('E_X2'), sym('E2_X')

In [37]:
var_x = E_X2 - E2_X

In [42]:
var_z = a * a * var_x + b**2 * var_y - 2 * a * b * cov_xy


In [41]:
var_z

var_z

In [43]:
var_z.subs([(a, 2), (b, 3), (var_x, 2), (var_y, 3.5), (cov_xy, -0.8)])


49.1000000000000